## Homogeneous Gas

Here is a notebook for homogeneous gas model.

Here we are talking about a homogeneous gas bulk of neutrinos with single energy. The EoM is
$$
i \partial_t \rho_E = \left[ \frac{\delta m^2}{2E}B +\lambda L +\sqrt 2 G_F \int_0^\infty dE' ( \rho_{E'} - \bar \rho_{E'} ) ,\rho_E \right]
$$

while the EoM for antineutrinos is
$$
i \partial_t \bar\rho_E = \left[- \frac{\delta m^2}{2E}B +\lambda L +\sqrt 2 G_F \int_0^\infty dE' ( \rho_{E'} - \bar \rho_{E'} ) ,\bar\rho_E \right]
$$

Initial:
Homogeneous, Isotropic, Monoenergetic $\nu_e$ and $\bar\nu_e$

The equations becomes
$$
i \partial_t \rho_E = \left[ \frac{\delta m^2}{2E} B +\lambda L +\sqrt 2 G_F   ( \rho_{E} - \bar \rho_{E} ) ,\rho_E \right]
$$
$$
i \partial_t \bar\rho_E = \left[- \frac{\delta m^2}{2E}B +\lambda_b L +\sqrt 2 G_F ( \rho_{E} - \bar \rho_{E} ) ,\bar\rho_E \right]
$$



Define $\omega=\frac{\delta m^2}{2E}$,  $\omega = \frac{\delta m^2}{-2E}$, $\mu=\sqrt{2}G_F n_\nu$
$$
i \partial_t \rho_E = \left[ \omega B +\lambda L +\mu   ( \rho_{E} - \bar \rho_{E} ) ,\rho_E \right]
$$
$$
i \partial_t \bar\rho_E = \left[\bar\omega B +\bar\lambda L +\mu ( \rho_{E} - \bar \rho_{E} ) ,\bar\rho_E \right]
$$


where

$$
B = \frac{1}{2} \begin{pmatrix} 
-\cos 2\theta_v & \sin 2\theta_v \\
\sin 2\theta_v & \cos 2\theta_v
\end{pmatrix} = 
\begin{pmatrix} 
-0.38729833462 & 0.31622776601\\
0.31622776601 & 0.38729833462
\end{pmatrix}
$$

$$
L = \begin{pmatrix}
1 & 0 \\
0 & 0
\end{pmatrix}
$$

Initial condition 
$$
\rho(t=0) = \begin{pmatrix}
1 & 0 \\
0 & 0
\end{pmatrix}
$$

$$
\bar\rho(t=0) =\begin{pmatrix}
1 & 0 \\
0 & 0
\end{pmatrix}
$$

define the following quantities

1. hbar$=\hbar$
2. delm2E$= \delta m^2/2E$
3. lamb $= \lambda$, lambb $= \bar\lambda$
4. gF $= G_F$
5. mu $=\mu$
6. omega $=\omega$, omegab $=\bar\omega$

## Numerical

In [305]:
# This line configures matplotlib to show figures embedded in the notebook, 
# instead of opening a new window for each figure. More about that later. 
# If you are using an old version of IPython, try using '%pylab inline' instead.
%matplotlib inline
%load_ext snakeviz

import numpy as np
from scipy.optimize import minimize
from scipy.special import expit
import matplotlib.pyplot as plt

from matplotlib.lines import Line2D

import timeit

import pandas as pd

import plotly.plotly as py
from plotly.graph_objs import *
import plotly.tools as tls

The snakeviz extension is already loaded. To reload it, use:
  %reload_ext snakeviz


In [306]:
# hbar=1.054571726*10**(-34)
hbar=1
delm2E=1
lamb=1  ## lambda for neutrinos
lambb=1 ## lambda for anti neutrinos
gF=1
nd=1  ## number density
ndb=1   ## number density
omega=1
omegab=1

## Here are some matrices to be used

elM = np.array([[1,0],[0,0]])
bM = 1/2*np.array( [ [ - 0.38729833462,0.31622776601] , [0.31622776601,0.38729833462] ] )

## sqareroot of 2
sqrt2=np.sqrt(2)

Using Mathematica, I can find the 4\*2 equations

In [307]:

#r11prime(t)

## The matrix eqn for neutrinos. Symplify the equation to the form A.X=0. Here I am only writing down the LHS.
## Eqn for r11'
# 1/2*( r21(t)*( bM12*delm2E - 2*sqrt2*gF*rb12(t) ) + r12(t) * ( -bM21*delm2E + 2*sqrt2*gF*rb21(t) ) - 1j*r11prime(t)  )
## Eqn for r12'
# 1/2*( r22(t)* ( bM12 ) )

### wait a minute I don't actually need to write down this. I can just do this part in numpy.

I am going to substitute all density matrix elements using their corrosponding network expressions.

So first of all, I need the network expression for the unknown functions.

A function is written as

$$ y_i= 1+t_i v_k f(t_i w_k+u_k) ,$$

while it's derivative is

$$v_k f(t w_k+u_k) + t v_k f(tw_k+u_k) (1-f(tw_k+u_k)) w_k .$$

Now I can write down the equations using these two forms.

In [308]:
def trigf(x):
    #return 1/(1+np.exp(-x)) # It's not bad to define this function here for people could use other functions other than expit(x).
    return expit(x)

In [309]:
## The time derivative part

### Here are the initial conditions

init = np.array( [[1,0],[0,0]] )
initb = np.array([[0,0],[0,0]])

### For neutrinos

def rho(x,ti,initialCondition): # x is the input structure arrays, ti is a time point

    v11,w11,u11,v12,w12,u12,v21,w21,u21,v22,w22,u22 = np.split(x,12)[:12]
        
    elem11= np.sum(ti * v11 * trigf( ti*w11 +u11 ) )
    elem12= np.sum(ti * v12 * trigf( ti*w12 +u12 ) )
    elem21= np.sum(ti * v21 * trigf( ti*w21 +u21 ) )
    elem22= np.sum(ti * v22 * trigf( ti*w22 +u22 ) )
    
    return initialCondition + np.array([[ elem11 , elem12 ],[elem21, elem22]])

def rhob(xb,ti,initialConditionb): # x is the input structure arrays, ti is a time point

    vb11,wb11,ub11,vb12,wb12,ub12,vb21,wb21,ub21,vb22,wb22,ub22 = np.split(xb,12)[:12]

    elem11= np.sum(ti * vb11 * trigf( ti*wb11 +ub11 ) )
    elem12= np.sum(ti * vb12 * trigf( ti*wb12 +ub12 ) )
    elem21= np.sum(ti * vb21 * trigf( ti*wb21 +ub21 ) )
    elem22= np.sum(ti * vb22 * trigf( ti*wb22 +ub22 ) )
    
    return initialConditionb + np.array([[ elem11 , elem12 ],[elem21, elem22]])



In [310]:
## Test
xtemp=np.ones(120)
rho(xtemp,1,init)

array([[ 9.80797078,  8.80797078],
       [ 8.80797078,  8.80797078]])

In [311]:
## Define Hamiltonians for both

def hamil(x,xb,ti,initialCondition,initialConditionb):
    
    return delm2E*bM + lamb*elM + sqrt2*gF*( rho(x,ti,initialCondition) - rhob(xb,ti,initialConditionb) )


def hamilb(x,xb,ti,initialCondition,initialConditionb):
    
    return -delm2E*bM + lambb*elM + sqrt2*gF*( rho(x,ti,initialCondition) - rhob(xb,ti,initialConditionb) )

In [312]:
## The commutator

def comm(x,xb,ti,initialCondition,initialConditionb):
    
    return np.dot(hamil(x,xb,ti,initialCondition,initialConditionb), rho(x,ti,initialCondition) ) - np.dot(rho(x,ti,initialCondition), hamil(x,xb,ti,initialCondition,initialConditionb) )

def commb(x,xb,ti,initialCondition,initialConditionb):
    
    return np.dot(hamilb(x,xb,ti,initialCondition,initialConditionb), rhob(xb,ti,initialConditionb) ) - np.dot(rhob(xb,ti,initialConditionb), hamilb(x,xb,ti,initialCondition,initialConditionb) )


In [313]:
## Test

print "neutrino\n",comm(xtemp,xtemp,1,init,initb)
print "antineutrino\n",commb(xtemp,xtemp,0.5,init,initb)

neutrino
[[  0.          21.26432251]
 [-21.26432251   0.        ]]
antineutrino
[[ 0.          9.86899694]
 [-9.86899694  0.        ]]


In [314]:
## The COST of the eqn set

def costTi(x,xb,ti,initialCondition,initialConditionb):
    
    v11,w11,u11,v12,w12,u12,v21,w21,u21,v22,w22,u22 = np.split(x,12)[:12]
    vb11,wb11,ub11,vb12,wb12,ub12,vb21,wb21,ub21,vb22,wb22,ub22 = np.split(xb,12)[:12]
    
    fvec11 = np.array(trigf(ti*w11 + u11) )  # This is a vector!!!
    fvec12 = np.array(trigf(ti*w12 + u12) )
    fvec21 = np.array(trigf(ti*w21 + u21) )
    fvec22 = np.array(trigf(ti*w22 + u22) )
    
    fvecb11 = np.array(trigf(ti*wb11 + ub11) )  # This is a vector!!!
    fvecb12 = np.array(trigf(ti*wb12 + ub12) )
    fvecb21 = np.array(trigf(ti*wb21 + ub21) )
    fvecb22 = np.array(trigf(ti*wb22 + ub22) )
    
    
    costi11= ( np.sum (v11*fvec11 + ti * v11* fvec11 * ( 1 -  fvec11  ) * w11 ) + 1j*  ( comm(x,xb,ti,initialCondition,initialConditionb)[0,0] )  )  
    costi12= ( np.sum (v12*fvec12 + ti * v12* fvec12 * ( 1 -  fvec12  ) * w12 ) + 1j*  ( comm(x,xb,ti,initialCondition,initialConditionb)[0,1] )  )  
    costi21= ( np.sum (v21*fvec21 + ti * v21* fvec21 * ( 1 -  fvec21  ) * w21 ) + 1j*  ( comm(x,xb,ti,initialCondition,initialConditionb)[1,0] )  )  
    costi22= ( np.sum (v22*fvec22 + ti * v22* fvec22 * ( 1 -  fvec22  ) * w22 ) + 1j*  ( comm(x,xb,ti,initialCondition,initialConditionb)[1,1] )  )  

    costbi11= ( np.sum (vb11*fvecb11 + ti * vb11* fvecb11 * ( 1 -  fvecb11  ) * wb11 ) + 1j*  ( commb(x,xb,ti,initialCondition,initialConditionb)[0,0] )  )  
    costbi12= ( np.sum (vb12*fvecb12 + ti * vb12* fvecb12 * ( 1 -  fvecb12  ) * wb12 ) + 1j*  ( commb(x,xb,ti,initialCondition,initialConditionb)[0,1] )  )  
    costbi21= ( np.sum (vb21*fvecb21 + ti * vb21* fvecb21 * ( 1 -  fvecb21  ) * wb21 ) + 1j*  ( commb(x,xb,ti,initialCondition,initialConditionb)[1,0] )  )  
    costbi22= ( np.sum (vb22*fvecb22 + ti * vb22* fvecb22 * ( 1 -  fvecb22  ) * wb22 ) + 1j*  ( commb(x,xb,ti,initialCondition,initialConditionb)[1,1] )  )  
    
    return (np.real(costi11))**2 + (np.real(costi12))**2+ (np.real(costi21))**2 +  (np.real(costi22))**2 + (np.real(costbi11))**2 + (np.real(costbi12))**2 +(np.real(costbi21))**2 + (np.real(costbi22))**2 + (np.imag(costi11))**2 + (np.imag(costi12))**2+ (np.imag(costi21))**2 +  (np.imag(costi22))**2 + (np.imag(costbi11))**2 + (np.imag(costbi12))**2 +(np.imag(costbi21))**2 + (np.imag(costbi22))**2    


In [315]:
costTi(xtemp,xtemp,0,init,initb)

427.55731631081869

In [316]:
## Calculate the total cost

def cost(xtot,t,initialCondition,initialConditionb):
    
    x,xb = np.split(xtot,2)[:2]

    t = np.array(t)
    
    costTotal = np.sum( costTList(x,xb,t,initialCondition,initialConditionb)  )
        
    return costTotal
    

def costTList(x,xb,t,initialCondition,initialConditionb):  ## This is the function WITHOUT the square!!! 
        
    t = np.array(t)
    
    costList = np.asarray([])
    
    for temp in t:
        tempElement = costTi(x,xb,temp,initialCondition,initialConditionb)
        costList = np.append(costList, tempElement)
        
    return np.array(costList)

    

In [317]:
ttemp = np.linspace(0,10)
print ttemp

[  0.           0.20408163   0.40816327   0.6122449    0.81632653
   1.02040816   1.2244898    1.42857143   1.63265306   1.83673469
   2.04081633   2.24489796   2.44897959   2.65306122   2.85714286
   3.06122449   3.26530612   3.46938776   3.67346939   3.87755102
   4.08163265   4.28571429   4.48979592   4.69387755   4.89795918
   5.10204082   5.30612245   5.51020408   5.71428571   5.91836735
   6.12244898   6.32653061   6.53061224   6.73469388   6.93877551
   7.14285714   7.34693878   7.55102041   7.75510204   7.95918367
   8.16326531   8.36734694   8.57142857   8.7755102    8.97959184
   9.18367347   9.3877551    9.59183673   9.79591837  10.        ]


In [318]:
ttemp = np.linspace(0,10)
print costTList(xtemp,xtemp,ttemp,init,initb)
print cost(xtemp,ttemp,init,initb)

[    427.55731631     576.4910937      853.37726656    1282.49988421
    1884.10392755    2673.91751306    3663.20708292    4859.23007651
    6265.9208822     7884.65985964    9715.0132345    11755.37565319
   14003.48530838   16456.80879099   19112.80902104   21969.11701801
   25023.62977467   28274.5546376    31720.41721704   35360.04611511
   39192.54430487   43217.25409161   47433.72030466   51841.65465775
   56440.90298664   61231.41622272   66213.22539729   71386.4206155
   76751.13372951   82307.52433125   88055.76864167   93996.05087228
  100128.55665759  106453.46819441  112970.96076627  119681.20037537
  126584.34224643  133680.53000561  140969.89537205  148452.5582301
  156128.62697564  163998.19905246  172061.36161212  180318.1922465
  188768.75975409  197413.12491109  206251.3412265   215283.45566646
  224509.50933817  233929.53812735]
987353.364647


## Minimization

Here is the minimization

In [353]:
tlin = np.linspace(0,5,11)
initGuess = np.ones(360)
# initGuess = np.random.rand(1,30)+2

costF = lambda x: cost(x,tlin,init,initb)

In [354]:
cost(initGuess,tlin,init,initb)

511348.2296578954

In [355]:
## %%snakeviz
# startCG = timeit.default_timer()
#costFResultCG = minimize(costF,initGuess,method="CG")
#stopCG = timeit.default_timer()

#print stopCG - startCG

#print costFResultCG

In [356]:
%%snakeviz
startSLSQP = timeit.default_timer()
costFResultSLSQP = minimize(costF,initGuess,method="SLSQP")
stopSLSQP = timeit.default_timer()

print stopSLSQP - startSLSQP

print costFResultSLSQP

8197.502882
  status: 9
 success: False
    njev: 101
    nfev: 36944
     fun: 12.083728144220865
       x: array([ 0.08164624,  0.43012195,  1.45167192, -0.07429276,  3.81335331,
        2.38620944, -2.71224165, -2.70947969,  0.01699589,  0.11788438,
        0.53879474, -0.48002277, -1.16710793,  2.76399066,  3.99148753,
       -0.7942903 , -0.96776519, -1.52651749, -0.70672809,  0.47898006,
       -0.60192634, -0.26145079, -0.39436723, -1.11957685, -1.04926951,
       -0.73935089,  0.15475872, -0.02159688, -0.78090796,  0.7786671 ,
        4.14479404,  3.91192217, -0.05821563,  2.95515106, -2.97534003,
        2.39186039,  4.20947383,  4.12070046,  3.44191178,  3.70526911,
        3.6088506 ,  4.50961264,  4.5060854 ,  1.95191652, -1.66995103,
        0.04085454,  0.04080483,  0.04064098,  0.04028499,  0.0395552 ,
        0.0391922 ,  0.04026285,  0.04044502,  0.04103574,  0.04095955,
        0.03960498,  0.03860272,  0.03897518,  0.03854319,  0.03880976,
        1.11626868,  1.1163

In [357]:
costFResultSLSQP.get('x')

array([ 0.08164624,  0.43012195,  1.45167192, -0.07429276,  3.81335331,
        2.38620944, -2.71224165, -2.70947969,  0.01699589,  0.11788438,
        0.53879474, -0.48002277, -1.16710793,  2.76399066,  3.99148753,
       -0.7942903 , -0.96776519, -1.52651749, -0.70672809,  0.47898006,
       -0.60192634, -0.26145079, -0.39436723, -1.11957685, -1.04926951,
       -0.73935089,  0.15475872, -0.02159688, -0.78090796,  0.7786671 ,
        4.14479404,  3.91192217, -0.05821563,  2.95515106, -2.97534003,
        2.39186039,  4.20947383,  4.12070046,  3.44191178,  3.70526911,
        3.6088506 ,  4.50961264,  4.5060854 ,  1.95191652, -1.66995103,
        0.04085454,  0.04080483,  0.04064098,  0.04028499,  0.0395552 ,
        0.0391922 ,  0.04026285,  0.04044502,  0.04103574,  0.04095955,
        0.03960498,  0.03860272,  0.03897518,  0.03854319,  0.03880976,
        1.11626868,  1.1163978 ,  1.11648663,  1.11689965,  1.11754447,
        1.11782391,  1.11712936,  1.11676186,  1.1161053 ,  1.11

In [358]:
np.savetxt('./assets/homogen/optimize_ResultSLSQP360.txt', costFResultSLSQP.get('x'), delimiter = ',')

## Functions

Find the solutions to each elements.

In [359]:
# costFResultSLSQPx = np.genfromtxt('./assets/homogen/optimize_ResultSLSQP.txt', delimiter = ',')

In [360]:
## The first element of neutrino density matrix
xresult = np.split(costFResultSLSQP.get('x'),2)[0]
xresultb = np.split(costFResultSLSQP.get('x'),2)[1]
#xresult = np.split(costFResultSLSQPx,2)[0]
#xresultb = np.split(costFResultSLSQPx,2)[1]
## print xresult11

plttlin=np.linspace(0,5,100)

pltdata11 = np.array([])
pltdata22 = np.array([])

for i in plttlin:
    pltdata11 = np.append(pltdata11 ,rho(xresult,i,init)[0,0] )
    
print pltdata11

for i in plttlin:
    pltdata22 = np.append(pltdata22 ,rho(xresult,i,init)[1,1] )
    
print pltdata22

print "----------------------------------------"

pltdatab11 = np.array([])
pltdatab22 = np.array([])

for i in plttlin:
    pltdatab11 = np.append(pltdatab11 ,rho(xresultb,i,init)[0,0] )
    
print pltdatab11

for i in plttlin:
    pltdatab22 = np.append(pltdatab22 ,rho(xresultb,i,init)[1,1] )
    
print pltdatab22

[ 1.          1.0115493   1.02113013  1.02879731  1.0346207   1.03868421
  1.04108457  1.04192989  1.04133808  1.03943507  1.03635309  1.03222879
  1.02720146  1.02141124  1.01499745  1.00809702  1.00084303  0.99336344
  0.98577993  0.97820694  0.97075085  0.96350934  0.95657085  0.95001428
  0.94390873  0.93831341  0.93327762  0.92884091  0.92503318  0.92187501
  0.91937794  0.91754487  0.91637042  0.91584143  0.91593743  0.91663111
  0.91788887  0.91967133  0.92193388  0.92462724  0.927698    0.93108917
  0.93474075  0.9385903   0.94257345  0.94662451  0.95067695  0.95466399
  0.95851909  0.96217651  0.96557178  0.9686422   0.97132737  0.97356958
  0.97531433  0.97651071  0.97711181  0.97707514  0.97636297  0.97494266
  0.97278696  0.96987434  0.96618917  0.96172199  0.95646969  0.95043563
  0.94362982  0.93606896  0.9277765   0.91878269  0.90912453  0.89884575
  0.8879967   0.87663427  0.86482172  0.85262853  0.84013019  0.82740794
  0.81454858  0.80164414  0.78879162  0.77609265  0

In [361]:
#np.savetxt('./assets/homogen/optimize_pltdatar11.txt', pltdata11, delimiter = ',')
#np.savetxt('./assets/homogen/optimize_pltdatar22.txt', pltdata22, delimiter = ',')

In [362]:
plt.figure(figsize=(20,12.36))
plt.ylabel('Time')
plt.xlabel('rho11')
plt.plot(plttlin,pltdata11,"b4-",label="rho11")
py.iplot_mpl(plt.gcf(),filename="HG-rho11")


# tls.embed("https://plot.ly/~emptymalei/73/")

In [363]:
plt.figure(figsize=(20,12.36))
plt.ylabel('Time')
plt.xlabel('rho22')
plt.plot(plttlin,pltdata22,"r4-",label="rho22")
py.iplot_mpl(plt.gcf(),filename="HG-rho22")

In [364]:
plt.figure(figsize=(20,12.36))
plt.ylabel('Time')
plt.xlabel('rhob11')
plt.plot(plttlin,pltdatab11,"b*-",label="rhob11")
py.iplot_mpl(plt.gcf(),filename="HG-rhob11")


In [365]:
plt.figure(figsize=(20,12.36))
plt.ylabel('Time')
plt.xlabel('rhob22')
plt.plot(plttlin,pltdatab11,"b*-",label="rhob22")
py.iplot_mpl(plt.gcf(),filename="HG-rhob22")


In [366]:
MMA_optmize_pltdata = np.genfromtxt('./assets/homogen/MMA_optmize_pltdata.txt', delimiter = ',')

plt.figure(figsize=(20,12.36))
plt.ylabel('MMArho11')
plt.xlabel('Time')
plt.plot(np.linspace(0,5,501),MMA_optmize_pltdata,"r-",label="MMArho11")
plt.plot(plttlin,pltdata11,"b4-",label="rho11")
py.iplot_mpl(plt.gcf(),filename="MMA-rho11")

## Practice

In [367]:
xtemp1 = np.arange(4)
xtemp1.shape = (2,2)
print xtemp1
xtemp1[0,1]
np.dot(xtemp1,xtemp1)
xtemp1[0,1]

[[0 1]
 [2 3]]


1